In [1]:
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
train = pd.read_csv("train_data.csv", parse_dates=['Timestamp'])
test = pd.read_csv("test_data.csv", parse_dates=['Timestamp'])

In [3]:
df = pd.merge(train, test, how='outer').sort_values(by=['userID','Timestamp']).reset_index(drop=True)

In [4]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


In [5]:
## col_name를 기준으로 mean, std, sum을 추가하는 함수.

def new_feature_answer(df, col_name:str, new_feature_name:str):
    
    grouped_df = df.groupby(col_name)
    
    mean_series = grouped_df.mean()['answerCode']
    std_series = grouped_df.std()['answerCode']
    sum_series = grouped_df.sum()['answerCode']
    
    
    series2mean = dict()
    for i, v in zip(mean_series.keys(), mean_series.values):
        series2mean[i] = v
        
    series2std = dict()
    for i, v in zip(std_series.keys(), std_series.values):
        series2std[i] = v
        
    series2sum = dict()
    for i, v in zip(sum_series.keys(), sum_series.values):
        series2sum[i] = v
        
    df[f'{new_feature_name}_ans_mean'] = df[col_name].map(series2mean)
    df[f'{new_feature_name}_ans_std'] = df[col_name].map(series2std)
    df[f'{new_feature_name}_ans_sum'] = df[col_name].map(series2sum)
    
    return df

In [6]:
## col_name를 기준으로 mean, std, sum을 추가하는 함수.

def new_feature_time(df, col_name:str, new_feature_name:str):
    
    grouped_df = df.groupby(col_name)
    
    mean_series = grouped_df.mean()['elap_time']
    std_series = grouped_df.std()['elap_time']
    sum_series = grouped_df.sum()['elap_time']
    median_series = grouped_df.median()['elap_time']
    
    
    series2mean = dict()
    for i, v in zip(mean_series.keys(), mean_series.values):
        series2mean[i] = v
        
    series2std = dict()
    for i, v in zip(std_series.keys(), std_series.values):
        series2std[i] = v
        
    series2sum = dict()
    for i, v in zip(sum_series.keys(), sum_series.values):
        series2sum[i] = v

    series2median = dict()
    for i, v in zip(median_series.keys(), median_series.values):
        series2median[i] = v
        
    df[f'{new_feature_name}_time_mean'] = df[col_name].map(series2mean)
    df[f'{new_feature_name}_time_std'] = df[col_name].map(series2std)
    df[f'{new_feature_name}_time_sum'] = df[col_name].map(series2sum)
    df[f'{new_feature_name}_time_median'] = df[col_name].map(series2median)
    
    return df

In [7]:
# 난이도 설정을 위한 ELO 사용
def ELO_function(df):
    def get_new_theta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return theta + learning_rate_theta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def get_new_beta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return beta - learning_rate_beta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def learning_rate_theta(nb_answers):
        return max(0.3 / (1 + 0.01 * nb_answers), 0.04)

    def learning_rate_beta(nb_answers):
        return 1 / (1 + 0.05 * nb_answers)

    def probability_of_good_answer(theta, beta, left_asymptote):
        return left_asymptote + (1 - left_asymptote) * sigmoid(theta - beta)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def estimate_parameters(answers_df, granularity_feature_name="assessmentItemID"):
        item_parameters = {
            granularity_feature_value: {"beta": 0, "nb_answers": 0}
            for granularity_feature_value in np.unique(
                answers_df[granularity_feature_name]
            )
        }
        student_parameters = {
            student_id: {"theta": 0, "nb_answers": 0}
            for student_id in np.unique(answers_df.userID)
        }

        print("Parameter estimation is starting...")

        for student_id, item_id, left_asymptote, answered_correctly in tqdm(
            zip(
                answers_df.userID.values,
                answers_df[granularity_feature_name].values,
                answers_df.left_asymptote.values,
                answers_df.answerCode.values,
            )
        ):
            theta = student_parameters[student_id]["theta"]
            beta = item_parameters[item_id]["beta"]

            item_parameters[item_id]["beta"] = get_new_beta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                item_parameters[item_id]["nb_answers"],
            )
            student_parameters[student_id]["theta"] = get_new_theta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                student_parameters[student_id]["nb_answers"],
            )

            item_parameters[item_id]["nb_answers"] += 1
            student_parameters[student_id]["nb_answers"] += 1

        print(f"Theta & beta estimations on {granularity_feature_name} are completed.")
        return student_parameters, item_parameters

    def gou_func(theta, beta):
        return 1 / (1 + np.exp(-(theta - beta)))

    df["left_asymptote"] = 0

    print(f"Dataset of shape {df.shape}")
    print(f"Columns are {list(df.columns)}")

    student_parameters, item_parameters = estimate_parameters(df)

    prob = [
        gou_func(student_parameters[student]["theta"], item_parameters[item]["beta"])
        for student, item in zip(df.userID.values, df.assessmentItemID.values)
    ]

    df["elo_prob"] = prob

    return df

In [8]:
def feature_engineering(df):
    # ELO 적용
    df = ELO_function(df)

    # user의 문제 정답수, 풀이수, 정답률을 시간순으로 누적해서 계산 / 전체 평균 정답률
    df['user_cum_correct'] = df.groupby('userID')['answerCode'].cumsum()
    df['user_cum_cnt'] = df.groupby('userID')['answerCode'].cumcount()+1
    df['user_acc'] = (df['user_cum_correct']/df['user_cum_cnt'])
    stu_groupby = df.groupby('userID').agg({
            'assessmentItemID': 'count',
            'answerCode': 'sum'
    		  })
    stu_groupby['user_mean'] = stu_groupby['answerCode'] / stu_groupby['assessmentItemID']
    stu_groupby = stu_groupby.reset_index()
    df = df.merge(stu_groupby[['userID','user_mean']], on='userID', how='left')

    # user별 풀어본 총 문제수(total_count)
    total_count = df.groupby('userID')[['testId']].count().rename(columns={"testId":"total_count"})
    df = pd.merge(df, total_count, on='userID', how='left')

    # test 별 정답의 mean, std, sum, user별 푼 종류수, 누적 개수
    df = new_feature_answer(df, 'testId', 'test')
    test_count = df.groupby('userID')[['testId']].nunique().rename(columns={"testId":"test_count"})
    df = pd.merge(df, test_count, on='userID', how='left')
    df["test_cum_correct"] = df.groupby(["userID", "testId"])['answerCode'].cumsum()
    df["test_cum_cnt"] = df.groupby(["userID", "testId"]).cumcount()+1
    df["test_acc"] = (df['test_cum_correct']/df['test_cum_cnt'])

    # tag 별 정답의 mean, std, sum, user별 푼 종류수, 누적 개수
    df = new_feature_answer(df, 'KnowledgeTag', 'tag')
    tag_count = df.groupby('userID')[['KnowledgeTag']].nunique().rename(columns={"KnowledgeTag":"tag_count"})
    df = pd.merge(df, tag_count, on='userID', how='left')
    df["tag_cum_correct"] = df.groupby(["userID", "KnowledgeTag"])['answerCode'].cumsum()
    df["tag_cum_cnt"] = df.groupby(["userID", "KnowledgeTag"]).cumcount()+1
    df["tag_acc"] = (df['tag_cum_correct']/df['tag_cum_cnt'])

    # test 대분류(prefix) 별 정답의 mean, std, sum, user별 푼 종류수, 누적 개수
    df['prefix'] = df.assessmentItemID.map(lambda x: int(x[2:3]))
    df = new_feature_answer(df, 'prefix', 'prefix')
    prefix_count = df.groupby('userID')[['prefix']].nunique().rename(columns={"prefix":"prefix_count"})
    df = pd.merge(df, prefix_count, on='userID', how='left')
    df["prefix_cum_correct"] = df.groupby(["userID", "prefix"])['answerCode'].cumsum()
    df["prefix_cum_cnt"] = df.groupby(["userID", "prefix"]).cumcount()+1
    df["prefix_acc"] = (df['prefix_cum_correct']/df['prefix_cum_cnt'])

    # assessmentID 별 정답의 mean, std, sum, user별 푼 종류수
    df = new_feature_answer(df, 'assessmentItemID', 'assess')
    assess_count = df.groupby('userID')[['assessmentItemID']].nunique().rename(columns={"assessmentItemID":"assess_count"})
    df = pd.merge(df, assess_count, on='userID', how='left')
    df["assess_cum_correct"] = df.groupby(["userID", "assessmentItemID"])['answerCode'].cumsum()
    df["assess_cum_cnt"] = df.groupby(["userID", "assessmentItemID"]).cumcount()+1
    df["assess_acc"] = (df['assess_cum_correct']/df['assess_cum_cnt'])

    # assessmentID 뒤 3자리(suffix) 정답의 mean, std, sum, user별 푼 종류수
    df['suffix'] = df.assessmentItemID.map(lambda x: int(x[-3:]))
    df = new_feature_answer(df, 'suffix', 'suffix')
    suffix_count = df.groupby('userID')[['suffix']].nunique().rename(columns={"suffix":"suffix_count"})
    df = pd.merge(df, suffix_count, on='userID', how='left')
    df["suffix_cum_correct"] = df.groupby(["userID", "suffix"])['answerCode'].cumsum()
    df["suffix_cum_cnt"] = df.groupby(["userID", "suffix"]).cumcount()+1
    df["suffix_acc"] = (df['suffix_cum_correct']/df['suffix_cum_cnt'])

    # weekday, month, day, hour 별 정답의 mean, std, sum
    df['weekday'] = df['Timestamp'].dt.weekday
    df['month'] = df['Timestamp'].dt.month
    df['day'] = df['Timestamp'].dt.day
    df['hour'] = df['Timestamp'].dt.hour
    df = new_feature_answer(df,'weekday', 'weekday' )
    df = new_feature_answer(df,'month', 'month' )
    df = new_feature_answer(df,'day', 'day' )
    df = new_feature_answer(df,'hour', 'hour' )

    # 시간관련 feature engineering
    # user가 각 문제를 푸는 데 걸린 시간
    solving_time = df[['userID', 'Timestamp']].groupby('userID').diff(periods=-1).fillna(pd.Timedelta(seconds=0))
    solving_time = solving_time['Timestamp'].apply(lambda x: x.total_seconds())
    df['elap_time'] = -solving_time
    df['elap_time'] = df['elap_time'].map(lambda x: int(x) if 0 < x <= 3600 else int(89))

    # 각 기본 feature에 대한 elap_time의 mean, std, sum, median, 상대 시간(중앙값 - 풀이 시간)
    df = new_feature_time(df, 'userID', 'user')
    df = new_feature_time(df, 'testId', 'test')
    df = new_feature_time(df, 'KnowledgeTag', 'tag')
    df = new_feature_time(df, 'prefix', 'prefix')
    df = new_feature_time(df, 'assessmentItemID', 'assess')
    df = new_feature_time(df, 'suffix', 'suffix')
    
    df['user_relatvie_time'] = df['user_time_median'] - df['elap_time']
    df['test_relatvie_time'] = df['test_time_median'] - df['elap_time']
    df['tag_relatvie_time'] = df['tag_time_median'] - df['elap_time']
    df['prefix_relatvie_time'] = df['prefix_time_median'] - df['elap_time']
    df['assess_relatvie_time'] = df['assess_time_median'] - df['elap_time']
    df['suffix_relatvie_time'] = df['suffix_time_median'] - df['elap_time']

    # 맞은 row와 틀린 row를 분리
    df_o = df[df['answerCode']==1]
    df_x = df[df["answerCode"]==0]
    basic_feats = ['userID','assessmentItemID',"testId",'answerCode','Timestamp','KnowledgeTag']

    user_o_elp = new_feature_time(df_o, 'userID', 'user_o')
    user_o_elp['user_o_relatvie_time'] = user_o_elp['user_o_time_median'] - user_o_elp['elap_time']
    df = pd.merge(df, user_o_elp[basic_feats+['user_o_time_mean', 'user_o_time_std', 'user_o_time_sum', 'user_o_time_median', 'user_o_relatvie_time']],
            on=basic_feats, how="left").fillna(0)
    user_x_elp = new_feature_time(df_x, 'userID', 'user_x')
    user_x_elp['user_x_relatvie_time'] = user_x_elp['user_x_time_median'] - user_x_elp['elap_time']
    df = pd.merge(df, user_x_elp[basic_feats+['user_x_time_mean', 'user_x_time_std', 'user_x_time_sum', 'user_x_time_median', 'user_x_relatvie_time']],
            on=basic_feats, how="left").fillna(0)

    test_o_elp = new_feature_time(df_o, 'testId', 'test_o')
    test_o_elp['test_o_relatvie_time'] = test_o_elp['test_o_time_median'] - test_o_elp['elap_time']
    df = pd.merge(df, test_o_elp[basic_feats+['test_o_time_mean', 'test_o_time_std', 'test_o_time_sum', 'test_o_time_median', 'test_o_relatvie_time']],
            on=basic_feats, how="left").fillna(0)
    test_x_elp = new_feature_time(df_x, 'testId', 'test_x')
    test_x_elp['test_x_relatvie_time'] = test_x_elp['test_x_time_median'] - test_x_elp['elap_time']
    df = pd.merge(df, test_x_elp[basic_feats+['test_x_time_mean', 'test_x_time_std', 'test_x_time_sum', 'test_x_time_median', 'test_x_relatvie_time']],
            on=basic_feats, how="left").fillna(0)

    tag_o_elp = new_feature_time(df_o, 'KnowledgeTag', 'tag_o')
    tag_o_elp['tag_o_relatvie_time'] = tag_o_elp['tag_o_time_median'] - tag_o_elp['elap_time']
    df = pd.merge(df, tag_o_elp[basic_feats+['tag_o_time_mean', 'tag_o_time_std', 'tag_o_time_sum', 'tag_o_time_median', 'tag_o_relatvie_time']],
            on=basic_feats, how="left").fillna(0)
    tag_x_elp = new_feature_time(df_x, 'KnowledgeTag', 'tag_x')
    tag_x_elp['tag_x_relatvie_time'] = tag_x_elp['tag_x_time_median'] - tag_x_elp['elap_time']
    df = pd.merge(df, tag_x_elp[basic_feats+['tag_x_time_mean', 'tag_x_time_std', 'tag_x_time_sum', 'tag_x_time_median', 'tag_x_relatvie_time']],
            on=basic_feats, how="left").fillna(0)

    prefix_o_elp = new_feature_time(df_o, 'prefix', 'prefix_o')
    prefix_o_elp['prefix_o_relatvie_time'] = prefix_o_elp['prefix_o_time_median'] - prefix_o_elp['elap_time']
    df = pd.merge(df, prefix_o_elp[basic_feats+['prefix_o_time_mean', 'prefix_o_time_std', 'prefix_o_time_sum', 'prefix_o_time_median', 'prefix_o_relatvie_time']],
            on=basic_feats, how="left").fillna(0)
    prefix_x_elp = new_feature_time(df_x, 'prefix', 'prefix_x')
    prefix_x_elp['prefix_x_relatvie_time'] = prefix_x_elp['prefix_x_time_median'] - prefix_x_elp['elap_time']
    df = pd.merge(df, prefix_x_elp[basic_feats+['prefix_x_time_mean', 'prefix_x_time_std', 'prefix_x_time_sum', 'prefix_x_time_median', 'prefix_x_relatvie_time']],
            on=basic_feats, how="left").fillna(0)

    assess_o_elp = new_feature_time(df_o, 'assessmentItemID', 'assess_o')
    assess_o_elp['assess_o_relatvie_time'] = assess_o_elp['assess_o_time_median'] - assess_o_elp['elap_time']
    df = pd.merge(df, assess_o_elp[basic_feats+['assess_o_time_mean', 'assess_o_time_std', 'assess_o_time_sum', 'assess_o_time_median', 'assess_o_relatvie_time']],
            on=basic_feats, how="left").fillna(0)
    assess_x_elp = new_feature_time(df_x, 'assessmentItemID', 'assess_x')
    assess_x_elp['assess_x_relatvie_time'] = assess_x_elp['assess_x_time_median'] - assess_x_elp['elap_time']
    df = pd.merge(df, assess_x_elp[basic_feats+['assess_x_time_mean', 'assess_x_time_std', 'assess_x_time_sum', 'assess_x_time_median', 'assess_x_relatvie_time']],
            on=basic_feats, how="left").fillna(0)

    suffix_o_elp = new_feature_time(df_o, 'suffix', 'suffix_o')
    suffix_o_elp['suffix_o_relatvie_time'] = suffix_o_elp['suffix_o_time_median'] - suffix_o_elp['elap_time']
    df = pd.merge(df, suffix_o_elp[basic_feats+['suffix_o_time_mean', 'suffix_o_time_std', 'suffix_o_time_sum', 'suffix_o_time_median', 'suffix_o_relatvie_time']],
            on=basic_feats, how="left").fillna(0)
    suffix_x_elp = new_feature_time(df_x, 'suffix', 'suffix_x')
    suffix_x_elp['suffix_x_relatvie_time'] = suffix_x_elp['suffix_x_time_median'] - suffix_x_elp['elap_time']
    df = pd.merge(df, suffix_x_elp[basic_feats+['suffix_x_time_mean', 'suffix_x_time_std', 'suffix_x_time_sum', 'suffix_x_time_median', 'suffix_x_relatvie_time']],
            on=basic_feats, how="left").fillna(0)
    
    # test 푸는데 걸린 시간 총합
    user_solve_test_time = df.groupby(['userID', 'testId'])['elap_time'].sum()\
                                                 .groupby(level=0).cumsum().reset_index()
    user_solve_test_time.rename(columns={"elap_time":"test_solve_time"}, inplace=True)
    df = df.merge(user_solve_test_time, on=['userID','testId'], how='left')

    # 현 시점 user의 학습 누적 시간
    df['total_used_time'] = df.groupby('userID')['elap_time'].cumsum()

    # user의 최근 3문제, 최근 5문제 문제풀이 평균시간
    df['recent3_elap_time'] = df.groupby(['userID'])['elap_time'].rolling(3).mean().fillna(0).values

    # user나 prefix가 바뀔때마다 시간 리셋
    time_df = df[["userID", "prefix", "Timestamp"]].sort_values(by=["userID", "prefix", "Timestamp"])
    time_df["userID_reset"] = time_df["userID"] != time_df["userID"].shift(1)
    time_df["prefix_reset"] = time_df["prefix"] != time_df["prefix"].shift(1)
    time_df["first"] = time_df[["userID_reset", "prefix_reset"]].any(axis=1).apply(lambda x: 1 - int(x))
    time_df["reset_time"] = time_df["Timestamp"].diff().fillna(pd.Timedelta(seconds=0))
    time_df["reset_time"] = (
        time_df["reset_time"].apply(lambda x: x.total_seconds()) * time_df["first"]
    )
    df["reset_time"] = time_df["reset_time"]#.apply(lambda x: math.log(x + 1))
    
    return df

In [28]:
df = pd.merge(train, test, how='outer').sort_values(by=['userID','Timestamp']).reset_index(drop=True)

In [9]:
df = feature_engineering(df)

Dataset of shape (2526700, 7)
Columns are ['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'left_asymptote']
Parameter estimation is starting...


2526700it [00:16, 156077.81it/s]


Theta & beta estimations on assessmentItemID are completed.


/tmp/ipykernel_35261/4058175095.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{new_feature_name}_time_mean'] = df[col_name].map(series2mean)
/tmp/ipykernel_35261/4058175095.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{new_feature_name}_time_std'] = df[col_name].map(series2std)
/tmp/ipykernel_35261/4058175095.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [17]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,left_asymptote,elo_prob,user_cum_correct,user_cum_cnt,...,suffix_o_relatvie_time,suffix_x_time_mean,suffix_x_time_std,suffix_x_time_sum,suffix_x_time_median,suffix_x_relatvie_time,test_solve_time,total_used_time,recent3_elap_time,reset_time
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,0.980768,1,1,...,28.0,0.000000,0.000000,0.0,0.0,0.0,2854,3,0.000000,-0.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,0.973315,2,2,...,24.0,0.000000,0.000000,0.0,0.0,0.0,2854,11,0.000000,3.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,0.947292,3,3,...,27.0,0.000000,0.000000,0.0,0.0,0.0,2854,18,6.000000,8.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,0.974914,4,4,...,31.0,0.000000,0.000000,0.0,0.0,0.0,2854,25,7.333333,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,0.961391,5,5,...,78.0,0.000000,0.000000,0.0,0.0,0.0,2854,36,8.333333,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,0,0.296188,1,5,...,0.0,105.562218,295.170555,16985172.0,89.0,0.0,309,309,49.000000,24.0
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0,0.616957,2,6,...,20.0,0.000000,0.000000,0.0,0.0,0.0,528,320,41.333333,0.0
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,0,0.601200,3,7,...,-14.0,0.000000,0.000000,0.0,0.0,0.0,528,366,48.666667,11.0
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,0,0.726027,4,8,...,-39.0,0.000000,0.000000,0.0,0.0,0.0,528,439,43.333333,46.0


In [22]:
df.columns.size

161

In [19]:
df.columns.tolist()

['userID',
 'assessmentItemID',
 'testId',
 'answerCode',
 'Timestamp',
 'KnowledgeTag',
 'left_asymptote',
 'elo_prob',
 'user_cum_correct',
 'user_cum_cnt',
 'user_acc',
 'user_mean',
 'total_count',
 'test_ans_mean',
 'test_ans_std',
 'test_ans_sum',
 'test_count',
 'test_cum_correct',
 'test_cum_cnt',
 'test_acc',
 'tag_ans_mean',
 'tag_ans_std',
 'tag_ans_sum',
 'tag_count',
 'tag_cum_correct',
 'tag_cum_cnt',
 'tag_acc',
 'prefix',
 'prefix_ans_mean',
 'prefix_ans_std',
 'prefix_ans_sum',
 'prefix_count',
 'prefix_cum_correct',
 'prefix_cum_cnt',
 'prefix_acc',
 'assess_ans_mean',
 'assess_ans_std',
 'assess_ans_sum',
 'assess_count',
 'assess_cum_correct',
 'assess_cum_cnt',
 'assess_acc',
 'suffix',
 'suffix_ans_mean',
 'suffix_ans_std',
 'suffix_ans_sum',
 'suffix_count',
 'suffix_cum_correct',
 'suffix_cum_cnt',
 'suffix_acc',
 'weekday',
 'month',
 'day',
 'hour',
 'weekday_ans_mean',
 'weekday_ans_std',
 'weekday_ans_sum',
 'month_ans_mean',
 'month_ans_std',
 'month_ans_su

In [23]:
train = df[df['answerCode']!=-1]
test = df[df['answerCode']==-1]

In [24]:
train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,left_asymptote,elo_prob,user_cum_correct,user_cum_cnt,...,suffix_o_relatvie_time,suffix_x_time_mean,suffix_x_time_std,suffix_x_time_sum,suffix_x_time_median,suffix_x_relatvie_time,test_solve_time,total_used_time,recent3_elap_time,reset_time
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,0.980768,1,1,...,28.0,0.000000,0.000000,0.0,0.0,0.0,2854,3,0.000000,-0.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,0.973315,2,2,...,24.0,0.000000,0.000000,0.0,0.0,0.0,2854,11,0.000000,3.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,0.947292,3,3,...,27.0,0.000000,0.000000,0.0,0.0,0.0,2854,18,6.000000,8.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,0.974914,4,4,...,31.0,0.000000,0.000000,0.0,0.0,0.0,2854,25,7.333333,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,0.961391,5,5,...,78.0,0.000000,0.000000,0.0,0.0,0.0,2854,36,8.333333,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,0,0.296188,1,5,...,0.0,105.562218,295.170555,16985172.0,89.0,0.0,309,309,49.000000,24.0
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0,0.616957,2,6,...,20.0,0.000000,0.000000,0.0,0.0,0.0,528,320,41.333333,0.0
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,0,0.601200,3,7,...,-14.0,0.000000,0.000000,0.0,0.0,0.0,528,366,48.666667,11.0
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,0,0.726027,4,8,...,-39.0,0.000000,0.000000,0.0,0.0,0.0,528,439,43.333333,46.0


In [25]:
test

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,left_asymptote,elo_prob,user_cum_correct,user_cum_cnt,...,suffix_o_relatvie_time,suffix_x_time_mean,suffix_x_time_std,suffix_x_time_sum,suffix_x_time_median,suffix_x_relatvie_time,test_solve_time,total_used_time,recent3_elap_time,reset_time
2989,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289,0,0.452318,716,1036,...,0.0,0.0,0.0,0.0,0.0,0.0,89412,91788,51.333333,46.0
3660,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080,0,0.484820,464,671,...,0.0,0.0,0.0,0.0,0.0,0.0,54263,54263,50.666667,23.0
10860,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660,0,0.184150,914,1317,...,0.0,0.0,0.0,0.0,0.0,0.0,105235,108532,33.000000,8.0
15278,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611,0,0.372931,1030,1260,...,0.0,0.0,0.0,0.0,0.0,0.0,165529,166498,90.000000,75.0
23531,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422,0,0.330752,292,387,...,0.0,0.0,0.0,0.0,0.0,0.0,23630,24050,40.666667,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525938,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615,0,0.076693,6,24,...,0.0,0.0,0.0,0.0,0.0,0.0,810,1033,31.000000,2.0
2526081,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636,0,0.624517,6,15,...,0.0,0.0,0.0,0.0,0.0,0.0,601,707,72.666667,107.0
2526282,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402,0,0.613140,6,15,...,0.0,0.0,0.0,0.0,0.0,0.0,359,650,40.666667,24.0
2526297,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402,0,0.396708,1,15,...,0.0,0.0,0.0,0.0,0.0,0.0,1530,1530,47.000000,21.0


## Feature Detail

### User
+ user_cum_correct: 유저의 현시점 누적 정답수
+ user_cum_cnt: 유저의 현시점 누적 문항수
+ user_acc: 유저의 현시점까지 정답률  
+ user_mean: 유저별 평균 정답률
+ total_count: 유저가 푼 총 문항수

<br>

### Test
+ test_ans_mean: 테스트별 평균 정답률
+ test_ans_sum: 테스트별 정답자 총합
+ test_ans_std: 테스트별 표준편차
+ test_count: 유저별 푼 테스트 종류수
+ test_cum_correct: 테스트의 누적 정답수
+ test_cum_cnt: 테스트의 누적 개수
+ test_acc: 테스트의 현시점까지 정답률

<br>

### Tag
+ KnowledgeTag: 원본 데이터의 KnowledgeTag
+ tag_ans_mean: 태그별 평균 정답률
+ tag_ans_sum: 태그별 정답자 총합
+ tag_ans_std: 태그별 표준편차
+ tag_count: 유저별 푼 문제 종류수
+ tag_cum_correct: 태그의 누적 정답수
+ tag_cum_cnt: 태그의 누적 개수
+ tag_acc: 태그의 현시점까지 정답률

<br>

### Prefix
+ prefix: 문항 대분류 A'XXX'0000000
+ prefix_ans_mean: 대분류 별 정답률
+ prefix_ans_std: 대분류 별 표준편차
+ prefix_ans_sum: 대분류 별 정답자 총합
+ prefix_count: 유저별 풀어 본 대분류 수
+ prefix_cum_correct: 대분류의 누적 정답수
+ prefix_cum_cnt: 대분류의 누적 개수
+ prefix_acc: 대분류의 현시점까지 정답률

<br>

### Asseseement
+ assess_ans_mean: 문항별 평균 정답률
+ assess_ans_std: 문항별 정답자 총합
+ assess_ans_sum: 문항별 표준표차
+ assess_count: 유저별 푼 태그 종류수
+ assess_cum_correct: 문항의 누적 정답수
+ assess_cum_cnt: 문항의 누적 개수
+ assess_acc: 문항의 현시점까지 정답률

<br>

### Suffix
+ suffix: 문항 뒷자리 A0000000'XXX'
+ suffix_ans_mean: 뒷자리 별 정답률
+ suffix_ans_std: 뒷자리 별 표준편차
+ suffix_ans_sum: 뒷자리 별 정답자 총합
+ suffix_count: 유저별 풀어 본 뒷자리 수
+ suffix_cum_correct: 뒷자리 누적 정답수
+ suffix_cum_cnt: 뒷자리 누적 개수
+ suffix_acc: 뒷자리 현시점까지 정답률

<br>

### Real Time
+ weekday: 문제를 푼 요일
+ weekday_ans_mean: 요일별 평균 정답률
+ weekday_ans_std: 요일별 표준편차
+ weekday_ans_sum: 요일별

<br>

+ month: 문제를 푼 월
+ month_ans_mean: 월별 평균 정답률
+ month_ans_std: 월별 표준편차
+ month_ans_sum: 월별 푼 문제 수

<br>

+ day: 문제를 푼 일
+ day_ans_mean: 일별 평균 정답률
+ day_ans_std: 일별 표준편차
+ day_ans_sum: 일별 푼 문제 수

<br>

+ hour: 문제를 푼 시간
+ hour_ans_mean: 시간별 평균 정답률
+ hour_ans_std: 시간별 표준편차
+ hour_ans_sum: 시간별 푼 문제수


<br>

### Elapsed Time
+ elap_time: 문항별 풀이에 걸린 시간

<br>

### user time
+ user_time_mean: 유저별 평균 풀이 시간
+ user_time_std: 유저별 풀이 시간의 표준편차
+ user_time_sum: 유저별 풀이 시간 총합
+ user_time_median: 유저별 풀이 시간의 중앙값
+ user_relatvie_time: user_time_median - elap_time 중앙값과 풀이 시간 차이

<br>

### test time
+ test_time_mean: 테스트별 평균 풀이 시간
+ test_time_std: 테스트별 풀이 시간의 표준편차
+ test_time_sum: 테스트별 풀이 시간 총합
+ test_time_median: 테스트별 풀이 시간의 중앙값
+ test_relatvie_time: test_time_median - elap_time 중앙값과 풀이 시간 차이

<br>

### tag time
+ tag_time_mean: 태그별 평균 풀이 시간
+ tag_time_std: 태그별 풀이 시간의 표준편차
+ tag_time_sum: 태그별 풀이 시간 총합
+ tag_time_median: 태그별 풀이 시간의 중앙값
+ tag_relatvie_time: tag_time_median - elap_time 중앙값과 풀이 시간 차이

<br>

### prefix time
+ prefix_time_mean: 대분류별 평균 풀이 시간
+ prefix_time_std: 대분류별 풀이 시간의 표준편차
+ prefix_time_sum: 대분류별 풀이 시간 총합
+ prefix_time_median: 대분류별 풀이 시간의 중앙값
+ prefix_relatvie_time: prefix_time_median - elap_time 중앙값과 풀이 시간 차이

<br>

### assess time
+ assess_time_mean: 문항별 평균 풀이 시간
+ assess_time_std: 문항별 풀이 시간의 표준편차
+ assess_time_sum: 문항별 풀이 시간 총합
+ assess_time_median: 문항별 풀이 시간의 중앙값
+ assess_relatvie_time: assess_time_median - elap_time 중앙값과 풀이 시간 차이

<br>

### suffix time
+ suffix_time_mean: 뒷자리별 평균 풀이 시간
+ suffix_time_std: 뒷자리별 풀이 시간의 표준편차
+ suffix_time_sum: 뒷자리별 풀이 시간 총합
+ suffix_time_median: 뒷자리별 풀이 시간의 중앙값
+ suffix_relatvie_time: suffix_time_median - elap_time 중앙값과 풀이 시간 차이

### user ox time
+ user_o_time_mean: 유저별 맞은 문제의 평균 풀이 시간
+ user_o_time_std: 유저별 맞은 문제의 풀이 시간의 표준편차
+ user_o_time_sum: 유저별 맞은 문제의 풀이 시간 총합
+ user_o_time_median: 유저별 맞은 문제의 풀이 시간의 중앙값
+ user_o_relatvie_time: user_o_time_median - elap_time
+ user_x_time_mean: 유저별 틀린 문제의 평균 풀이 시간
+ user_x_time_std: 유저별 틀린 문제의 풀이 시간의 표준편차
+ user_x_time_sum: 유저별 틀린 문제의 풀이 시간 총합
+ user_x_time_median: 유저별 틀린 문제의 풀이 시간의 중앙값
+ user_x_relatvie_time: user_x_time_median - elap_time

<br>

### test ox time
+ test_o_time_mean: 테스트별 맞은 문제의 평균 풀이 시간
+ test_o_time_std: 테스트별 맞은 문제의 풀이 시간의 표준편차
+ test_o_time_sum: 테스트별 맞은 문제의 풀이 시간 총합
+ test_o_time_median: 테스트별 맞은 문제의 풀이 시간의 중앙값
+ test_o_relatvie_time: test_o_time_median - elap_time
+ test_x_time_mean: 테스트별 틀린 문제의 평균 풀이 시간
+ test_x_time_std: 테스트별 틀린 문제의 풀이 시간의 표준편차
+ test_x_time_sum: 테스트별 틀린 문제의 풀이 시간 총합
+ test_x_time_median: 테스트별 틀린 문제의 풀이 시간의 중앙값
+ test_x_relatvie_time: test_x_time_median - elap_time

<br>

### tag ox time
+ tag_o_time_mean: 태그별 맞은 문제의 평균 풀이 시간
+ tag_o_time_std: 태그별 맞은 문제의 풀이 시간의 표준편차
+ tag_o_time_sum: 태그별 맞은 문제의 풀이 시간 총합
+ tag_o_time_median: 태그별 맞은 문제의 풀이 시간의 중앙값
+ tag_o_relatvie_time: tag_o_time_median - elap_time
+ tag_x_time_mean: 태그별 틀린 문제의 평균 풀이 시간
+ tag_x_time_std: 태그별 틀린 문제의 풀이 시간의 표준편차
+ tag_x_time_sum: 태그별 틀린 문제의 풀이 시간 총합
+ tag_x_time_median: 태그별 틀린 문제의 풀이 시간의 중앙값
+ tag_x_relatvie_time: tag_x_time_median - elap_time

<br>

### prefix ox time
+ prefix_o_time_mean: 대분류별 맞은 문제의 평균 풀이 시간
+ prefix_o_time_std: 대분류별 맞은 문제의 풀이 시간의 표준편차
+ prefix_o_time_sum: 대분류별 맞은 문제의 풀이 시간 총합
+ prefix_o_time_median: 대분류별 맞은 문제의 풀이 시간의 중앙값
+ prefix_o_relatvie_time: prefix_o_time_median - elap_time
+ prefix_x_time_mean: 대분류별 틀린 문제의 평균 풀이 시간
+ prefix_x_time_std: 대분류별 틀린 문제의 풀이 시간의 표준편차
+ prefix_x_time_sum: 대분류별 틀린 문제의 풀이 시간 총합
+ prefix_x_time_median: 대분류별 틀린 문제의 풀이 시간의 중앙값
+ prefix_x_relatvie_time: prefix_x_time_median - elap_time

<br>

### assessment ox time
+ assess_o_time_mean: 문항별 맞은 문제의 평균 풀이 시간
+ assess_o_time_std: 문항별 맞은 문제의 풀이 시간의 표준편차
+ assess_o_time_sum: 문항별 맞은 문제의 풀이 시간 총합
+ assess_o_time_median: 문항별 맞은 문제의 풀이 시간의 중앙값
+ assess_o_relatvie_time: assess_o_time_median - elap_time
+ assess_x_time_mean: 문항별 틀린 문제의 평균 풀이 시간
+ assess_x_time_std: 문항별 틀린 문제의 풀이 시간의 표준편차
+ assess_x_time_sum: 문항별 틀린 문제의 풀이 시간 총합
+ assess_x_time_median: 문항별 틀린 문제의 풀이 시간의 중앙값
+ assess_x_relatvie_time: assess_x_time_median - elap_time

<br>

### suffix ox time
+ suffix_o_time_mean: 뒷자리별 맞은 문제의 평균 풀이 시간
+ suffix_o_time_std: 뒷자리별 맞은 문제의 풀이 시간의 표준편차
+ suffix_o_time_sum: 뒷자리별 맞은 문제의 풀이 시간 총합
+ suffix_o_time_median: 뒷자리별 맞은 문제의 풀이 시간의 중앙값
+ suffix_o_relatvie_time: suffix_o_time_median - elap_time
+ suffix_x_time_mean: 뒷자리별 틀린 문제의 평균 풀이 시간
+ suffix_x_time_std: 뒷자리별 틀린 문제의 풀이 시간의 표준편차
+ suffix_x_time_sum: 뒷자리별 틀린 문제의 풀이 시간 총합
+ suffix_x_time_median: 뒷자리별 틀린 문제의 풀이 시간의 중앙값
+ suffix_x_relatvie_time: suffix_x_time_median - elap_time

<br>

### ETC
+ test_solve_time: test 푸는데 걸린 시간 총합의 누적합
+ total_used_time: 현 시점 user의 학습 누적 시간
+ recent3_elap_time: 최근 푼 3문제의 평균 풀이 시간
+ reset_time: user나 prefix가 바뀔때마다 시간 리셋

In [ ]:
# 유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
df = add_last_problem(df)

df["mid"] = df["testId"].apply(lambda x: int(x[-3:]))
df["problem_number"] = df["assessmentItemID"].apply(lambda x: int(x[-3:]))


df["GradeElp"] = (
    df.groupby(["userID", "grade"])["elapsed"]
    .transform(lambda x: x.cumsum())
    .fillna(0)
)
df["GradeMElp"] = df["GradeElp"] / [
    v if v != 0 else 1 for v in df["GradeCount"].values
]

f = lambda x: len(set(x))
test = df.groupby(["testId"]).agg({"problem_number": "max", "KnowledgeTag": f})
test.reset_index(inplace=True)

test.columns = ["testId", "problem_count", "tag_count"]

df = pd.merge(df, test, on="testId", how="left")

df["problem_position"] = df["problem_number"] / df["problem_count"]
df["solve_order"] = df.groupby(["userID", "testId"]).cumcount()
df["solve_order"] = (
    df["solve_order"]
    - df["problem_count"] * (df["solve_order"] > df["problem_count"]).apply(int)
    + 1
)
df["retest"] = (df["solve_order"] > df["problem_count"]).apply(int)
T = df["solve_order"] != df["problem_number"]
TT = T.shift(1)
TT[0] = False
df["solved_disorder"] = (TT.apply(lambda x: not x) & T).apply(int)

df["testId"] = df["testId"].apply(lambda x: int(x[1:4] + x[-3]))
